In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Div
from bokeh.layouts import row

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
display(HTML("<style>.container { width:100% !important; }</style>"))
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')

Loading BokehJS ...

Backtesting of a UNI v2 LUNC/BUSD LP strategy
---


In [2]:
from cpy_amm.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

In [3]:
from binance import Client
from cpy_amm.utils import format_df
from cpy_amm.plotting import new_trade_figure, plot_price_ratio

mkt_pair = 'LUNC/BUSD'
start, end = '2022-10-01 00:00:00', '2023-04-25 23:59:59'
frequency = Client.KLINE_INTERVAL_1HOUR

df_trades = client.get_trade_data(mkt_pair, frequency, start, end)
df_trades["quantity"] = 0.0001*df_trades["quantity"]
show(Div(text=format_df(df_trades.head(10), width=800)))

________________________________________________________________________________
[Memory] Calling cpy_amm.binance_loader.get_trade_data...
get_trade_data('LUNC/BUSD', '1h', '2022-10-01 00:00:00', '2023-04-25 23:59:59')


Fetching klines from Binance:   0%|          | 0/10 [00:00<?, ?it/s]

___________________________________________________get_trade_data - 1.8s, 0.0min


In [4]:
from cpy_amm.market import MarketQuote, MarketPair, new_market, Pool, with_mkt_price
from cpy_amm.simulation import swap_simulation
from cpy_amm.plotting import new_simulation_figure, cp_amm_autoviz

liquidity_usd = 1000000
# LUNC/BUSD market price
base = MarketQuote("LUNC/USD", df_trades.price.iloc[0])
# LUNC/BUSD market price
quote = MarketQuote("BUSD/USD", 1)
# create a 100000 USD market for LUNC/USTC
mkt = new_market(liquidity_usd, quote, base, 0.003)
# run simulation
simul = swap_simulation(mkt, df_trades, is_arb_enabled=True)
# display sim results
show(new_simulation_figure(mkt, simul, plot_height=300))
# show(Div(text=format_df(simul["breakdown"].head(100), width=3200)))

Function 'trade_summary' executed in 0.0107s
Function 'sim_results' executed in 0.0488s
Function 'swap_simulation' executed in 0.3603s
Function 'new_pnl_figure' executed in 0.0252s
Function 'new_portfolio_figure' executed in 0.0396s
Function 'new_price_figure' executed in 0.0758s
Function 'new_fitted_pnl_figure' executed in 0.0421s
Function 'new_price_impact_figure' executed in 0.0238s
Function 'new_roi_distrib_figure' executed in 0.0148s
Function 'new_pnl_arb_figure' executed in 0.0250s


In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

simul["breakdown"].head(100)

,side,arb_profit,price,price_impact,mid_price,mkt_price,spread,avg_price,current_base,current_quote,cp_invariant,total_fees_paid_quote,total_volume_base,total_volume_quote,asset_base_pct,hold_portfolio,current_portfolio,trade_pnl,total_pnl,roi,impermanent_loss,mkt_price_ratio,volume_base,volume_quote,fees_paid_quote,trade_pnl_pct,fees_pnl_pct,total_arb_profit,asset_quote_pct
trade_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-10-01 00:00:00,sell,0.00000,0.00030,0.00027,0.00030,0.00030,0.00000,0.00030,"1,653,883,419.91401","499,865.68991","826,719,576,719,576.75000",0.40414,"444,266.47485",-134.71424,0.50000,"999,731.82003","999,731.37981",-0.44022,-0.03608,-0.00000,0.00000,1.00000,"444,266.47485",-134.71424,0.40414,-0.00000,0.00000,0.00000,0.50000
2022-10-01 01:00:00,sell,10.92628,0.00030,0.00778,0.00030,0.00030,-0.00000,0.00030,"1,666,849,146.91835","495,977.44238","826,719,576,719,576.75000",12.10398,"13,409,993.47919","-4,034.66161",0.50000,"991,999.35068","991,954.88476",-44.46592,-32.36194,-0.00003,-0.00003,0.98393,"12,965,727.00434","-3,899.94737",11.69984,-0.00004,0.00001,10.92628,0.50000
2022-10-01 01:00:00,sell,0.00000,0.00030,0.00017,0.00030,0.00030,0.00000,0.00030,"1,667,134,324.61530","495,892.60116","826,719,576,719,576.75000",12.35927,"13,695,171.17615","-4,119.75811",0.50000,"991,831.30305","991,785.20232",-46.10072,-33.74145,-0.00003,-0.00003,0.98393,"285,177.69695",-85.09651,0.25529,-0.00005,0.00001,10.92628,0.50000
2022-10-01 02:00:00,buy,0.00000,0.00030,-0.00004,0.00030,0.00030,0.00000,0.00030,"1,667,060,388.64896","495,914.59455","826,719,576,719,576.75000",12.42545,"13,621,235.20981","-4,097.69855",0.50000,"991,874.86326","991,829.18909",-45.67417,-33.24872,-0.00003,-0.00001,0.99011,"-73,935.96634",22.05956,0.06618,-0.00005,0.00001,10.92628,0.50000
2022-10-01 03:00:00,buy,0.00000,0.00030,-0.00001,0.00030,0.00030,0.00000,0.00030,"1,667,036,513.74147","495,921.69692","826,719,576,719,576.75000",12.44682,"13,597,360.30231","-4,090.57481",0.50000,"991,888.93067","991,843.39384",-45.53684,-33.09001,-0.00003,-0.00002,0.98647,"-23,874.90749",7.12374,0.02137,-0.00005,0.00001,10.92628,0.50000
2022-10-01 04:00:00,sell,4.19272,0.00030,0.00632,0.00029,0.00029,-0.00000,0.00030,"1,677,632,108.59104","492,789.55290","826,719,576,719,576.87500",21.87153,"24,192,955.15188","-7,232.14353",0.50000,"985,704.78332","985,579.10580",-125.67753,-103.80599,-0.00011,-0.00011,0.97133,"10,595,594.84957","-3,141.56873",9.42471,-0.00013,0.00002,15.11900,0.50000
2022-10-01 04:00:00,sell,0.00000,0.00029,0.00005,0.00029,0.00029,0.00000,0.00030,"1,677,712,741.92434","492,765.86871","826,719,576,719,576.87500",21.94280,"24,273,588.48519","-7,255.89899",0.50000,"985,658.17043","985,531.73742",-126.43301,-104.49021,-0.00011,-0.00011,0.97133,"80,633.33330",-23.75545,0.07127,-0.00013,0.00002,15.11900,0.50000
2022-10-01 05:00:00,sell,0.00000,0.00029,0.00010,0.00029,0.00029,-0.00000,0.00030,"1,677,875,509.52099","492,718.06641","826,719,576,719,576.87500",22.08664,"24,436,356.08184","-7,303.84513",0.50000,"985,564.09746","985,436.13281",-127.96465,-105.87801,-0.00011,-0.00017,0.96389,"162,767.59665",-47.94614,0.14384,-0.00013,0.00002,15.11900,0.50000
2022-10-01 06:00:00,buy,0.00000,0.00029,-0.00008,0.00029,0.00029,0.00000,0.00030,"1,677,734,353.10118","492,759.52131","826,719,576,719,576.75000",22.21137,"24,295,199.66203","-7,262.26549",0.50000,"985,645.67848","985,519.04262",-126.63586,-104.42449,-0.00011,-0.00008,0.97540,"-141,156.41981",41.57964,0.12474,-0.00013,0.00002,15.11900,0.50000
